In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#Loaded Saved Model
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BBILAP_weightVintage_60_in_15mob.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [3]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [4]:
combined_data = model_data.copy()

In [5]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [6]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [7]:
df = combined_data.dropna(subset=['60_in_15mob'])

In [8]:
df.shape

(123367, 2831)

In [9]:
df['user_segment'].value_counts()

user_segment
OFFUS    90704
ONUS     32663
Name: count, dtype: int64

In [10]:
#Take out onus record
df_onus = df[df['user_segment']=='ONUS']

In [11]:
df_onus.shape

(32663, 2831)

In [12]:
#take out offus records only
df_offus= df[df['user_segment']=='OFFUS']

In [13]:
df_offus.shape

(90704, 2831)

In [14]:
#change the data records (df_offus - offus , df_onus - onus)
data = df_offus.copy()

In [15]:
data.shape

(90704, 2831)

In [16]:
import numpy as np
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [18]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [19]:
model_df1.shape, oot_df1.shape

((74732, 2831), (13115, 2831))

In [20]:
features_list = [
'num_PERSONAL LOAN_enq_last_6M',
'num_CONSUMER LOAN_enq_last_6M_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'num_CREDIT CARD_enq_last_12M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_emi',
'total_credit_all_open',
    
# banking
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

#new banking features
'ratio_balance_after_2_days_max_credit_M3',
'ratio_5th_day_balance_to_max_credit_M3',
                 
#Loan application Features
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',
    
    
#ID Features
'aggs907',
'dm216s',
'dm206s',
'balmag01',
'g406s',

# New BUreau features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
'month_year',
'60_in_15mob']

In [21]:
model_df = model_df1[features_list]

In [22]:
oot_df = oot_df1[features_list]

In [23]:
model_df.shape, oot_df.shape

((74732, 40), (13115, 40))

In [24]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [25]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [26]:
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3037517/3109673050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [27]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3037517/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [28]:
X_oot.shape,y_oot.shape

((13115, 38), (13115,))

In [29]:
# Split Model dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [30]:
X_train.shape, X_test.shape

((44839, 38), (29893, 38))

In [31]:
# Create DMatrix for Train predictions
data_xtra_pred = xgb.DMatrix(X_train)
X_train['60_in_15mob'] = y_train
X_train['y_pred'] = loaded_model.predict(data_xtra_pred)

In [32]:
data_xt_pred = xgb.DMatrix(X_test)
X_test['60_in_15mob']= y_test
X_test['y_pred'] = loaded_model.predict(data_xt_pred)

In [33]:
# Create DMatrix for OOT predictions
data_x_pred = xgb.DMatrix(X_oot)
X_oot['60_in_15mob'] = y_oot
X_oot['y_pred'] = loaded_model.predict(data_x_pred)

In [34]:
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, 'y_pred', '60_in_15mob')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, 'y_pred', '60_in_15mob')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, 'y_pred', '60_in_15mob')

In [35]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")

Train AUC: 0.71, Gini: 0.42, KS: 0.31
Test AUC: 0.71, Gini: 0.41, KS: 0.30
OOT AUC: 0.71, Gini: 0.43, KS: 0.32


In [36]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.305640         0.593571        0.848376        0.504355   1312   
9          0.193745         0.452748        0.504313        0.411794   1311   
8          0.129573         0.377746        0.411781        0.348666   1312   
7          0.106026         0.324601        0.348615        0.302739   1311   
6          0.092296         0.284890        0.302679        0.268639   1311   
5          0.077744         0.252654        0.268622        0.237073   1312   
4          0.063310         0.222295        0.237046        0.207077   1311   
3          0.045732         0.191884        0.207065        0.177116   1312   
2          0.045004         0.163061        0.177092        0.148736   1311   
1          0.031250         0.127637        0.148699        0.069447   1312   

        label_good  label_bad  
Decile                         
10             911        401  
9             1057        254  
8             1142        170  
7             1172        139  
6             1190        121  
5             1210        102  
4             1228         83  
3             1252         60  
2             1252         59  
1             1271         41

In [37]:
feature_importance = loaded_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
35                                         days_on_pg       277.0
25                                              g406s       211.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       173.0
30                           No of enq in past 90days       154.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...       118.0
26                         percentage_ever_delinquent       116.0
31                          Num_Enq_non_BANK_last_12M       105.0
19  deviation_from_median_income/standard_deviatio...       100.0
6                highest_sanction_amount_credit_cards        94.0
4                                     Utilization_all        92.0
21                                            aggs907        92.0
11                         bounced_transactions_count        90.0
1                   num_CONSUMER LOAN_enq_last_6M_36M        90.0
12                           Min_Balance_Min_M1_M2_M3        88.0
9         

In [38]:
feature_importance_df

Feature  Importance
35                                         days_on_pg       277.0
25                                              g406s       211.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       173.0
30                           No of enq in past 90days       154.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...       118.0
26                         percentage_ever_delinquent       116.0
31                          Num_Enq_non_BANK_last_12M       105.0
19  deviation_from_median_income/standard_deviatio...       100.0
6                highest_sanction_amount_credit_cards        94.0
4                                     Utilization_all        92.0
21                                            aggs907        92.0
11                         bounced_transactions_count        90.0
1                   num_CONSUMER LOAN_enq_last_6M_36M        90.0
12                           Min_Balance_Min_M1_M2_M3        88.0
9                                           total_emi        85.0
34  MEAN(raw.amount WHERE device_os_category = Win...        81.0
24                                           balmag01        81.0
7                          Age_of_oldest_open_account        80.0
8             Utilization_CREDIT CARD_opened_last_36M        78.0
3                        num_CREDIT CARD_enq_last_12M        77.0
33                                  distinct_merchant        68.0
22                                             dm216s        64.0
13                               num_credit_gt_salary        58.0
37                              amt_first_android_txn        56.0
28           months_since_most_recent_delinquency_all        54.0
23                                             dm206s        52.0
10                              total_credit_all_open        47.0
32                     Num_of_installment_loan_opened        44.0
20                loan_amount_to_income#last_365_Days        39.0
17           ratio_balance_after_2_days_max_credit_M3        39.0
18             ratio_5th_day_balance_to_max_credit_M3        37.0
36  NUM_UNIQUE(pg_success_raw_data.merchant_id WHE...        35.0
27   months_since_most_recent_delinquency_installment        28.0
29                       percentage_utilization_gt_75        28.0
14                                    upi_trans_count        21.0
16               ratio_total_debit_to_total_credit_M3        18.0
15                            Max_Credit_Avg_M1_M2_M3        17.0
0                       num_PERSONAL LOAN_enq_last_6M        13.0